# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [155]:
df = pd.read_csv('tic-tac-toe.csv')

df_clean = pd.get_dummies(df)

predictors = df_clean.iloc[:, :-1]  
target = df_clean.iloc[:, -1]

scaler = StandardScaler()
predictors = predictors.astype(int)  
predictors_scaled = predictors
#predictors_scaled = scaler.fit_transform(predictors)
#predictors_scaled = pd.DataFrame(predictors_scaled, columns=predictors.columns)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [156]:
# your code here
from sklearn.model_selection import train_test_split

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [157]:
from __future__ import print_function

import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

from keras.optimizers import Adam

batch_size = 64
num_classes = 10
epochs = 20
learning_rate = 0.001 

model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(predictors.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              #optimizer=RMSprop(),
              optimizer = Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

/Users/weemoe/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [158]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(X_test, y_test))

Epoch 1/20
11/11 - 4s - 343ms/step - accuracy: 0.1687 - loss: 2.3312 - val_accuracy: 0.4097 - val_loss: 2.1364
Epoch 2/20
11/11 - 0s - 8ms/step - accuracy: 0.4000 - loss: 2.0694 - val_accuracy: 0.5486 - val_loss: 1.9139
Epoch 3/20
11/11 - 0s - 7ms/step - accuracy: 0.5224 - loss: 1.8570 - val_accuracy: 0.5694 - val_loss: 1.6718
Epoch 4/20
11/11 - 0s - 7ms/step - accuracy: 0.5522 - loss: 1.6041 - val_accuracy: 0.5764 - val_loss: 1.3881
Epoch 5/20
11/11 - 0s - 9ms/step - accuracy: 0.5776 - loss: 1.3221 - val_accuracy: 0.5764 - val_loss: 1.1006
Epoch 6/20
11/11 - 0s - 11ms/step - accuracy: 0.5701 - loss: 1.0831 - val_accuracy: 0.6250 - val_loss: 0.8719
Epoch 7/20
11/11 - 0s - 8ms/step - accuracy: 0.6015 - loss: 0.8698 - val_accuracy: 0.6667 - val_loss: 0.7308
Epoch 8/20
11/11 - 0s - 7ms/step - accuracy: 0.6403 - loss: 0.7675 - val_accuracy: 0.7431 - val_loss: 0.6509
Epoch 9/20
11/11 - 0s - 7ms/step - accuracy: 0.6582 - loss: 0.7127 - val_accuracy: 0.8021 - val_loss: 0.6039
Epoch 10/20
11/1

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [159]:
# your code here
import numpy as np
y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis=1)

comparison_df = pd.DataFrame({
    'Actual': y_test.values,  # Convert y_test from series to array
    'Predicted': y_pred_class
})

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [160]:
print(comparison_df.sample(10)) 

     Actual  Predicted
48    False          0
181    True          1
267   False          0
251   False          0
35    False          0
9      True          1
97    False          0
212   False          0
226   False          0
120    True          1


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [161]:
# your code here
from __future__ import print_function

import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
#from tensorflow.keras.optimizers import RMSprop

from keras.optimizers import Adam

batch_size = 64
num_classes = 10
epochs = 100
learning_rate = 0.001 

model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(predictors.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              #optimizer=RMSprop(),
              optimizer = Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

/Users/weemoe/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [162]:
# your answer here
history2 = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(X_test, y_test))

Epoch 1/100


11/11 - 3s - 303ms/step - accuracy: 0.4090 - loss: 2.1478 - val_accuracy: 0.4132 - val_loss: 2.0457
Epoch 2/100
11/11 - 0s - 11ms/step - accuracy: 0.4791 - loss: 1.9075 - val_accuracy: 0.4201 - val_loss: 1.7792
Epoch 3/100
11/11 - 0s - 33ms/step - accuracy: 0.5045 - loss: 1.5996 - val_accuracy: 0.4236 - val_loss: 1.4221
Epoch 4/100
11/11 - 0s - 12ms/step - accuracy: 0.5388 - loss: 1.2376 - val_accuracy: 0.5729 - val_loss: 1.0362
Epoch 5/100
11/11 - 0s - 8ms/step - accuracy: 0.5881 - loss: 0.9135 - val_accuracy: 0.6979 - val_loss: 0.7630
Epoch 6/100
11/11 - 0s - 8ms/step - accuracy: 0.6388 - loss: 0.7374 - val_accuracy: 0.6910 - val_loss: 0.6434
Epoch 7/100
11/11 - 0s - 8ms/step - accuracy: 0.6597 - loss: 0.6568 - val_accuracy: 0.7326 - val_loss: 0.5764
Epoch 8/100
11/11 - 0s - 8ms/step - accuracy: 0.6746 - loss: 0.6166 - val_accuracy: 0.7674 - val_loss: 0.5237
Epoch 9/100
11/11 - 0s - 9ms/step - accuracy: 0.7403 - loss: 0.5542 - val_accuracy: 0.8438 - val_loss: 0.4653
Epoch 10/100
11/1